In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_generator=ImageDataGenerator(rescale=1/255)
eval_generator=ImageDataGenerator(rescale=1/255)

In [ ]:
height = 256
width = 256
batch_sizes = 128
train_data_generator=train_generator.flow_from_directory('../input/face-mask-12k-images-dataset/Face Mask Dataset/Train',
                                             target_size=(height, width),
                                             batch_size = batch_sizes,
                                            shuffle = True,
                                            class_mode='binary')

In [ ]:
val_data_generator=eval_generator.flow_from_directory('../input/face-mask-12k-images-dataset/Face Mask Dataset/Validation',
                                             target_size=(height, width),
                                             batch_size = batch_sizes,
                                            shuffle = False,
                                            class_mode='binary')

In [ ]:
test_data_generator=eval_generator.flow_from_directory('../input/face-mask-12k-images-dataset/Face Mask Dataset/Test',
                                             target_size=(height, width),
                                             batch_size = batch_sizes,
                                            shuffle = False,
                                             class_mode='binary')

In [ ]:
train_data_generator.class_indices

In [ ]:
img_shape=(256,256,3)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,BatchNormalization,Dropout,MaxPool2D,Flatten,Dense

In [ ]:
model=Sequential()
model.add(Conv2D(filters=64,kernel_size=(2,2),input_shape=img_shape,activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(filters=128,kernel_size=(2,2),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(filters=256,kernel_size=(2,2),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(filters=512,kernel_size=(2,2),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense((512),activation='relu'))
model.add(Dropout(0.2))
model.add(Dense((256),activation='relu'))
model.add(Dropout(0.2))
model.add(Dense((128),activation='relu'))
model.add(Dropout(0.2))
model.add(Dense((64),activation='relu'))
model.add(Dropout(0.2))
model.add(Dense((32),activation='relu'))
model.add(Dropout(0.2))
model.add(Dense((1),activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs = 10
batch_size = 128
history = model.fit_generator(train_data_generator,
                                           steps_per_epoch = (train_data_generator.samples)//batch_size,
                                           epochs = epochs,
                                           validation_data = val_data_generator,
                                           validation_steps=(val_data_generator.samples)//batch_size )

In [ ]:
model.save('/kaggle/working/mask_model.h5')

In [ ]:
loss=pd.DataFrame(model.history.history)
loss.plot()

In [ ]:
#preds=model.predict_generator(test_data_generator)

In [ ]:
model.evaluate(test_data_generator)

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/kaggle/working/mask_model.h5')

In [ ]:
model.evaluate(test_data_generator)

In [ ]:
!pip install mtcnn
import mtcnn
from mtcnn import MTCNN
detector = MTCNN()

In [ ]:
from matplotlib import pyplot as plt
test_img_path = '../input/face-mask-12k-images-dataset/Face Mask Dataset/Test/WithMask/1175.png'
img = cv2.imread(test_img_path,cv2.IMREAD_COLOR)
plt.imshow(img[:,:,::-1])